In [1]:
import pathlib
from tensorflow import keras
import tensorflow as tf

In [2]:
# Scarichiamo e salviamo il dataset Stack Overflow.
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"
data_dir = keras.utils.get_file(origin=dataset_url,
                            extract=True,
                            cache_subdir='datasets/stack_overflow')
# Selezioniamo la cartella dove sono presenti i dati. 
# Di solito è ~\user\username\.keras\dataset.
# Da notare che questo dataset ha al suo interno due cartelle,
# una di train ed una di test. Nel nostro esercizio utilizzeremo
# soltanto quella di train.
data_dir = pathlib.Path(data_dir).parent
train_dir = pathlib.Path(data_dir, 'train')

In [3]:
BATCH_SIZE = 32
SEED = 42

# Creiamo i datset di training e validazione. Usiamo un validation
# split di 0.2 (quindi con rapporto 80/20).
train = keras.utils.text_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset='training',
    batch_size=BATCH_SIZE,
    seed=SEED)

val = keras.utils.text_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset='validation',
    batch_size=BATCH_SIZE,
    seed=SEED)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [4]:
# Mostriamo a schermo le classi presenti nel dataset usando
# l'attributo class_names.
print(f'Classi nel dataset: {train.class_names}')

Classi nel dataset: ['csharp', 'java', 'javascript', 'python']


In [5]:
# Creiamo un layer di tipo TextVectorization.
VOCAB_SIZE = 1000
vectorize_layer = keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')
# Chiamiamo il metodo adapt sul testo da caratterizzare.
train_text = train.map(lambda text, _: text)
vectorize_layer.adapt(train_text)

In [7]:
# Integriamo il layer di vettorizzazione all'interno di un
# modello di tipo sequenziale.
model = keras.models.Sequential()
model.add(
    keras.Input(shape=(1,),
    dtype=tf.string))
model.add(
    vectorize_layer)
model.add(
    keras.layers.Dense(
        64,
        activation='relu',
        name='dense_1'))
model.add(
    keras.layers.Dense(
        4,
        activation='softmax',
        name='dense_2'))

model.compile(optimizer='adam',
            loss=keras.losses.SparseCategoricalCrossentropy(),
            metrics=keras.metrics.SparseCategoricalAccuracy())

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_sparse_categorical_accuracy',
        min_delta=0.01,
        patience=3,
        restore_best_weights=True),
    keras.callbacks.TensorBoard()
]

history=model.fit(
    train,
    validation_data=val,
    epochs=10)

Epoch 1/10
200/200 [==============================] - 8s 30ms/step - loss: 0.9128 - sparse_categorical_accuracy: 0.6344 - val_loss: 0.6131 - val_sparse_categorical_accuracy: 0.7688
Epoch 2/10
200/200 [==============================] - 4s 18ms/step - loss: 0.5395 - sparse_categorical_accuracy: 0.7948 - val_loss: 0.5351 - val_sparse_categorical_accuracy: 0.7912
Epoch 3/10
200/200 [==============================] - 4s 19ms/step - loss: 0.4386 - sparse_categorical_accuracy: 0.8339 - val_loss: 0.5254 - val_sparse_categorical_accuracy: 0.7894
Epoch 4/10
200/200 [==============================] - 4s 20ms/step - loss: 0.3818 - sparse_categorical_accuracy: 0.8598 - val_loss: 0.5420 - val_sparse_categorical_accuracy: 0.7881
Epoch 5/10
200/200 [==============================] - 4s 21ms/step - loss: 0.3389 - sparse_categorical_accuracy: 0.8742 - val_loss: 0.5594 - val_sparse_categorical_accuracy: 0.7831
Epoch 6/10
200/200 [==============================] - 5s 25ms/step - loss: 0.3012 - sparse_cate